In [1]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows',1000)

In [2]:
#Read in merged paint data file
df = pd.read_csv('Paint_Data_Merged.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df.head(30)

,metric,value,time,is_prod,count
0,E_C100,390.0,2022-04-15 00:00:00,0.0,0.0
1,E_C100,120.0,2022-04-15 01:00:00,0.0,0.0
2,E_C100,120.0,2022-04-15 02:00:00,0.0,0.0
3,E_C100,120.0,2022-04-15 03:00:00,0.0,0.0
4,E_C100,110.0,2022-04-15 04:00:00,0.0,0.0
5,E_C100,110.0,2022-04-15 05:00:00,0.0,0.0
6,E_C100,100.0,2022-04-15 06:00:00,0.0,0.0
7,E_C100,110.0,2022-04-15 07:00:00,0.0,0.0
8,E_C100,110.0,2022-04-15 08:00:00,0.0,0.0
9,E_C100,110.0,2022-04-15 09:00:00,0.0,0.0


In [3]:
df.index = pd.to_datetime(df['time'], format='%Y.%m.%d %H:%M:%S')

In [4]:
df['metric'].unique()

array(['E_C100', 'E_C1000', 'E_C1100', 'E_C1200', 'E_C200', 'E_C300',
       'E_C400', 'E_C500', 'E_C600', 'E_C700', 'E_C800'], dtype=object)

In [5]:
meters = ['E_C100', 'E_C200', 'E_C300', 'E_C400', 'E_C500', 'E_C600', 'E_C700', 'E_C800']

In [6]:
def df_to_X_y_multivar(df, X_start = [], y_start = [], window_size = 10):
    df_as_np= df.to_numpy()
    for i in range(len(df_as_np)-window_size):
        row = [r for r in df_as_np[i:i+window_size]]
        X.append(row)
        label = df_as_np[i+window_size][0]
        y.append(label)
    return X, y 

In [7]:
df

,metric,value,time,is_prod,count
time,,,,,
2022-04-15 00:00:00,E_C100,390.0,2022-04-15 00:00:00,0.0,0.0
2022-04-15 01:00:00,E_C100,120.0,2022-04-15 01:00:00,0.0,0.0
2022-04-15 02:00:00,E_C100,120.0,2022-04-15 02:00:00,0.0,0.0
2022-04-15 03:00:00,E_C100,120.0,2022-04-15 03:00:00,0.0,0.0
2022-04-15 04:00:00,E_C100,110.0,2022-04-15 04:00:00,0.0,0.0
...,...,...,...,...,...
2022-06-29 06:00:00,E_C800,300.0,2022-06-29 06:00:00,0.0,63.0
2022-06-29 07:00:00,E_C800,310.0,2022-06-29 07:00:00,1.0,71.0
2022-06-29 08:00:00,E_C800,300.0,2022-06-29 08:00:00,1.0,56.0


In [8]:
#from sklearn import preprocessing
#min_max_scaler = preprocessing.MinMaxScaler()
#x_scaled = min_max_scaler.fit_transform(df['value'])

In [9]:
usage_df = df

usage_df['Seconds'] = df.index.map(pd.Timestamp.timestamp)

day = 60*60*24
year=day *365.2425

usage_df['Day sin'] = np.sin(usage_df['Seconds'] * (2*np.pi/day))
usage_df['Day cos'] = np.cos(usage_df['Seconds'] * (2*np.pi/day))
usage_df['Year sin'] = np.sin(usage_df['Seconds'] * (2*np.pi/year))
usage_df['Year cos'] = np.cos(usage_df['Seconds'] * (2*np.pi/year))

usage_df = usage_df.drop(['Seconds'], axis =1)
usage_df = usage_df.loc[usage_df['metric'].isin(meters)]

usage_df = pd.get_dummies(usage_df, columns=['metric'])
usage_df = usage_df.drop(['time'], axis = 1)

In [10]:
usage_df

,value,is_prod,count,Day sin,Day cos,Year sin,Year cos,metric_E_C100,metric_E_C200,metric_E_C300,metric_E_C400,metric_E_C500,metric_E_C600,metric_E_C700,metric_E_C800
time,,,,,,,,,,,,,,,
2022-04-15 00:00:00,390.0,0.0,0.0,-1.271365e-11,1.000000e+00,0.974794,-0.223108,1,0,0,0,0,0,0,0
2022-04-15 01:00:00,120.0,0.0,0.0,2.588190e-01,9.659258e-01,0.974634,-0.223807,1,0,0,0,0,0,0,0
2022-04-15 02:00:00,120.0,0.0,0.0,5.000000e-01,8.660254e-01,0.974473,-0.224505,1,0,0,0,0,0,0,0
2022-04-15 03:00:00,120.0,0.0,0.0,7.071068e-01,7.071068e-01,0.974312,-0.225204,1,0,0,0,0,0,0,0
2022-04-15 04:00:00,110.0,0.0,0.0,8.660254e-01,5.000000e-01,0.974150,-0.225902,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-29 06:00:00,300.0,0.0,63.0,1.000000e+00,1.302874e-11,0.051263,-0.998685,0,0,0,0,0,0,0,1
2022-06-29 07:00:00,310.0,1.0,71.0,9.659258e-01,-2.588190e-01,0.050547,-0.998722,0,0,0,0,0,0,0,1
2022-06-29 08:00:00,300.0,1.0,56.0,8.660254e-01,-5.000000e-01,0.049832,-0.998758,0,0,0,0,0,0,0,1


In [11]:
metric_cols = ['metric_E_C100','metric_E_C200','metric_E_C300','metric_E_C400','metric_E_C500','metric_E_C600','metric_E_C700','metric_E_C800']
WINDOW_SIZE= 24
X = []
y = []

for meter in metric_cols:
    df_subset = usage_df.loc[usage_df[meter]==1].copy()
    
    X, y= df_to_X_y_multivar(df_subset, X, y, WINDOW_SIZE) 

    print(len(X))
X = np.array(X)
y = np.array(y)

    

1787
3574
5361
7148
8935
10722
12509
14296


In [12]:
#Split arrays into random train and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

usage_training_mean = np.mean(X_train[:,:,0])
usage_training_std = np.std(X_train[:,:,0])

print('mean: ',usage_training_mean )
print('std: ',usage_training_std )

def preprocess(X_temp):
    X_temp[:,:,0] = (X_temp[:,:,0] - usage_training_mean) / usage_training_std
    return X_temp

def preprocess_output(y_temp):
    y_temp = (y_temp - usage_training_mean) / usage_training_std
    return y_temp

def postprocess_output(y_temp):
    y_temp = (y_temp* usage_training_std ) + usage_training_mean
    return y_temp


pp_X_train = preprocess(X_train)
pp_y_train = preprocess_output(y_train)

pp_X_test = preprocess(X_test)
pp_y_test = preprocess_output(y_test)

mean:  361.1588040690218
std:  299.8906283956299


In [13]:
usage_training_mean

361.1588040690218

In [14]:
 
test = np.array([1060, 1040, 1040])    


test = preprocess_output(test)


(test* usage_training_std ) + usage_training_mean

array([1060., 1040., 1040.])

<h1>Model</h1>

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

#model2 = Sequential()
#model2.add(InputLayer((WINDOW_SIZE, 15)))
#model2.add(GRU(64))
#model2.add(Dense(8,'relu'))
#model2.add(Dense(1,'linear'))

#model2.summary()



model = Sequential()
model.add(SimpleRNN(64, return_sequences=True, activation='swish', input_shape=(WINDOW_SIZE, 15)))
model.add(SimpleRNN(128, return_sequences=True, activation='swish'))
model.add(SimpleRNN(64, return_sequences=False, activation='swish'))
#Add dense layer
model.add(Dense(64, activation='swish'))
model.add(Dense(32, activation='swish'))
model.add(Dense(1,'linear'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 24, 64)            5120      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 24, 128)           24704     
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 64)                12352     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 48,449
Trainable params: 48,449
Non-traina

In [16]:
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.005), metrics=[RootMeanSquaredError()])

In [ ]:
model.fit(pp_X_train, pp_y_train, validation_data=(pp_X_test, pp_y_test), epochs=(100))

Epoch 1/100
358/358 [==============================] - 32s 64ms/step - loss: 0.8863 - root_mean_squared_error: 0.9414 - val_loss: 0.1370 - val_root_mean_squared_error: 0.3701
Epoch 2/100
358/358 [==============================] - 22s 62ms/step - loss: 0.1101 - root_mean_squared_error: 0.3318 - val_loss: 0.1006 - val_root_mean_squared_error: 0.3172
Epoch 3/100
358/358 [==============================] - 19s 54ms/step - loss: 0.0953 - root_mean_squared_error: 0.3087 - val_loss: 0.1088 - val_root_mean_squared_error: 0.3298
Epoch 4/100
358/358 [==============================] - 24s 68ms/step - loss: 0.0855 - root_mean_squared_error: 0.2924 - val_loss: 0.0967 - val_root_mean_squared_error: 0.3110
Epoch 5/100
358/358 [==============================] - 20s 55ms/step - loss: 0.0774 - root_mean_squared_error: 0.2783 - val_loss: 0.0929 - val_root_mean_squared_error: 0.3048
Epoch 6/100
180/358 [==============>...............] - ETA: 8s - loss: 0.0787 - root_mean_squared_error: 0.2804

In [ ]:
pp_X_test.shape

In [ ]:
test_predictions = model.predict(pp_X_test).flatten()
test_results=pd.DataFrame(data={'Test Predictions':postprocess_output(test_predictions), 'Actuals': y_test})
test_results.head(100)

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, postprocess_output(test_predictions), squared=False)

In [ ]:
test_predictions

In [ ]:
train_predictions = model.predict(preprocess(X)).flatten()
train_results=pd.DataFrame(data={'Train Predictions':postprocess_output(train_predictions), 'Actuals': y})
train_results.head(100)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(900,15))
plt.plot(train_results['Train Predictions'])
plt.plot(train_results['Actuals'])

<h1>Validation Set and Bootstrapping</h1>

In [ ]:
WINDOW_SIZE = 24


df_val = pd.read_csv('Val_Paint_Data_Merged.csv')
df_val = df_val.drop(['Unnamed: 0'], axis=1)
df_val.index = pd.to_datetime(df_val['time'], format='%Y.%m.%d %H:%M:%S')
df_val.head(30)

usage_df_val = df_val

usage_df_val['Seconds'] = df_val.index.map(pd.Timestamp.timestamp)

day = 60*60*24
year=day *365.2425

usage_df_val['Day sin'] = np.sin(usage_df_val['Seconds'] * (2*np.pi/day))
usage_df_val['Day cos'] = np.cos(usage_df_val['Seconds'] * (2*np.pi/day))
usage_df_val['Year sin'] = np.sin(usage_df_val['Seconds'] * (2*np.pi/year))
usage_df_val['Year cos'] = np.cos(usage_df_val['Seconds'] * (2*np.pi/year))

usage_df_val = usage_df_val.drop(['Seconds'], axis =1)
usage_df_val = usage_df_val.loc[usage_df_val['metric'].isin(meters)]

#make dummy variables for the values in column 'metric'
usage_df_val = pd.get_dummies(usage_df_val, columns=['metric'])
#Drop 'time' column
usage_df_val = usage_df_val.drop(['time'], axis = 1)

preds = []
acts = []

start_point = 100

for i in range(24*7):
    bootstrap_x = np.expand_dims(usage_df_val.to_numpy()[i + start_point :i + start_point + WINDOW_SIZE], 0)
    #print(bootstrap_x)
    pred_y = postprocess_output(model.predict(preprocess(bootstrap_x)))[0][0]
    act_y = usage_df_val.to_numpy()[start_point + i+WINDOW_SIZE][0]
    usage_df_val.iloc[start_point + i+WINDOW_SIZE, 0] = pred_y
    
    preds.append(pred_y)
    acts.append(act_y)
    
    
    #print(pred_y)
    #print(act_y)
    #print('--------')

In [ ]:
import matplotlib.pyplot as plt

val_results=pd.DataFrame(data={'Train Predictions':preds, 'Actuals': acts})


plt.figure(figsize=(10,5))
plt.plot(val_results['Train Predictions'], label='pred')
plt.plot(val_results['Actuals'], label='act')
plt.legend(loc="upper left")


In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(acts, preds, squared=False)

In [ ]:
import matplotlib.pyplot as plt

val_results=pd.DataFrame(data={'Train Predictions':preds, 'Actuals': acts})


plt.figure(figsize=(10,5))
plt.plot(val_results['Train Predictions'], label='pred')
plt.plot(val_results['Actuals'], label='act')
plt.legend(loc="upper left")
